In [16]:
import sys
import numpy as np
import matplotlib as plt
from nnfs.datasets import spiral_data

## Dataset Creation

In [17]:
#Create dataset with 100 data points for 3 classes of spirals
#Results in 300 total data points for classification
X, y = spiral_data(100, 3)

## Hidden Layers

In [18]:
#Creating hidden layers
class Layer_Dense:
    #Initialize using the number of inputs and neurons
    def __init__(self, n_inputs, n_neurons):
        #Create an array of size (n_inputs x n_neurons) that are random based on the normal distrution
        #scaled by a tenth
        self.w = 0.1 * np.random.randn(n_inputs, n_neurons)
        #Create a row vector based on the number of neurons
        self.bias = np.zeros([1, n_neurons])
    #Method to compute the output, takes in an input matrix
    def forward(self, inputs):
        self.output = inputs@self.w + self.bias

## Activation function

In [19]:
#Implementation of ReLU function        
class Activation_ReLU:
    def forward(self, inputs):
        self.output = np.maximum(0, inputs)
        
class Activation_Softmax:
    def forward(self, input):
        exp_val = np.exp(input - np.max(input, axis=1, keepdims=True))
        probability = exp_val/np.sum(exp_val, axis=1, keepdims=True)
        
        self.output = probability

The Rectified Linear Activation function is a piecewise function that  can be demonstrated as the following:

\begin{align*}
    &\text{If } x>0,     &  x\\
    &\text{If } x\leq 0, &  0\\
\end{align*}

This is important because it is a fast compututation that allows for the fitting of a non-linear signal given several neurons. This achieved by using a combination of the soft clipping for negative values and linear maping for natural numbers.

## 1st Hidden Layer

In [20]:
#Create the first hidden layer with 2 input and 3 output features
layer1 = Layer_Dense(2,3)
activation1 = Activation_ReLU()
layer1.forward(X)
activation1.forward(layer1.output)

## 2nd Hidden Layer

In [21]:
#Create second hidden layer with 3 input and 3 output features
layer2 = Layer_Dense(3,3)
activation2 = Activation_Softmax()
layer2.forward(activation1.output)
activation2.forward(layer2.output)

In [22]:
print(activation2.output.shape)

(300, 3)
